# vite

## hmr

In [ ]:
{
    /*
        ws msg type
    */
    
    let payload = {
        type:null,
        updates:[{
            type:null,
            path:null,
            timestamp:null
        }],
        event:null,
        data:null,
        paths:null,
        err:null,
    }
    switch(payload.type){
        case 'connected':
            setInterval(()=>ws.send('ping'),1000) // 保持 ws keep alive;
            break;
        case 'update':
            if(isFirst && hasError){
                window.location.reload()
            }else{
                clearError();
            }
            
            payload.updates.forEach(item=>{
                if(item.type == 'js-update'){
                    // 如果更新的是js文件
                    queueUpdate(fetchUpdate(item))
                }else{
                    // 如果是css文件
                    link.href = item.path + '?v='+ item.timestamp;
                    // 更新link的 href地址
                }
                
            })
            break;
        case 'custom':
            const cbs = customListenersMap.get(payload.event);
            if(cbs){
                cbs.forEach(cb=>cb(payload.data))
            }
            break;
        case 'full-reload':
            window.location.reload();
            break;
        case 'prune':
            // 清理残留的无用modules 逻辑，消除影响。side effects;
            payload.paths.forEach(path=>{
                const fn = pruneMap.get(path);
                if(fn){
                    fn(dataMap.get(path))
                }
            })
            break;
        case 'error':
            showError();
            break;
        default:
            return payload;
    }

}